In [7]:
import torch.nn as nn
import torch
import torch.optim
from torchvision.utils import save_image #保存图片
from torchvision.datasets import CIFAR10,MNIST #下载图片数据集
from torch.utils.data import DataLoader #读取批次
import torchvision.transforms as transforms #张量转换
from torch.autograd import Variable
import time #计时

In [14]:
# 读取数据集
dataset = MNIST(
    root='./data', train=True, transform = transforms.ToTensor(), download = True
)
# dataset = CIFAR10(root = './data', 
#                  download = True, transform = transforms.ToTensor()) #下载数据集
dataloader = DataLoader(dataset, batch_size= 64, shuffle= True)      # 按批次读取数据(一批64张，总共有50000张，所以有50000/64=781批)，shuffle= True打乱数据
#### 查看数据 ####
a = iter(dataloader)
print(next(a)[0].shape)
print(next(a)[1])   # 标签


torch.Size([64, 1, 28, 28])
tensor([6, 9, 8, 9, 5, 3, 1, 6, 0, 1, 5, 3, 1, 4, 7, 9, 2, 7, 6, 0, 7, 3, 9, 4,
        4, 7, 5, 2, 9, 8, 8, 9, 2, 8, 8, 2, 8, 2, 5, 5, 0, 6, 0, 5, 7, 5, 3, 3,
        4, 5, 7, 9, 5, 5, 3, 9, 9, 4, 3, 5, 6, 5, 1, 8])


In [18]:
# 构建判别网络
n_d_feature = 64 # 潜在大小
n_channel = 1 # 输入通道数
dnet = nn.Sequential(
        nn.Conv2d(n_channel, n_d_feature, kernel_size=4,
                 stride=2, padding=1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(n_d_feature, 2 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(2 * n_d_feature, 4 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(4 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(4 * n_d_feature, 1, kernel_size=3)).cuda()
print(dnet)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(3, 3), stride=(1, 1))
)


In [19]:
# 构建生成网络
latent_size = 64 # 输入通道数
n_g_feature = 64 # 输出通道数

gnet = nn.Sequential(
        nn.ConvTranspose2d(latent_size, 4 * n_g_feature, kernel_size=4,
                          bias=False),
        nn.BatchNorm2d(4 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(4 * n_g_feature, 2 * n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(2 * n_g_feature, n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(n_g_feature, n_channel, kernel_size=2,
                          stride=2, padding=2),
        nn.Tanh()).cuda()
print(gnet)

Sequential(
  (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): ConvTranspose2d(64, 1, kernel_size=(2, 2), stride=(2, 2), padding=(2, 2))
  (10): Tanh()
)


In [20]:
# 网络初始化
import torch.nn.init as init
def weights_init(m):
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)       # 指定模型权重进行赋值“用一个均匀分布生成值，填充输入的张量或变量”
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)   # 从给定均值和标准差的正态分布N(mean, std)中生成值，填充输入的张量或变量
        init.constant_(m.bias, 0)
        
gnet.apply(weights_init)
dnet.apply(weights_init)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(3, 3), stride=(1, 1))
)

## 函数解释
- torch.load('xxx.pth', map_location=lambda storage, loc: storage.cuda(0))：如果当初保存参数的时候是保存的cpu的参数，那么需要加载到GPU上则需要使用map_location来进行转化。
- torch.load('xxx.pth', map_location=lambda storage, loc: storage): 原本保存参数在GPU上，现在需要加载到CPU上。
- torch.load('xxx.pth', map_location={'cuda:1':'cuda:0'}): 原本保存参数在GPU1上，现在需要加载到GPU0上。

In [21]:
## 载入cpu训练的预参数
# checkpoint_d = torch.load('discriminator.pth', map_location=lambda storage, loc: storage.cuda(0))
# checkpoint_g = torch.load('generator.pth', map_location=lambda storage, loc: storage.cuda(0))
# dnet.load_state_dict(checkpoint_d)
# gnet.load_state_dict(checkpoint_g)

In [22]:
#载入gpu训练的预参数
#checkpoint_d = torch.load('D.pth')
#checkpoint_g = torch.load('G.pth')
#dnet.load_state_dict(checkpoint_d)
#gnet.load_state_dict(checkpoint_g)

In [23]:
#定义损失
criterion = nn.BCEWithLogitsLoss().cuda()
#定义优化器
goptimizer = torch.optim.Adam(gnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))

In [24]:
# 生成固定的噪音数据，输入到G网络的数据，用于检测生成网络的结果
batch_size = 64
fixed_noise = torch.randn(batch_size, latent_size, 1, 1).cuda()
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [25]:
#开始训练
start = time.time() #开始时间

epoch_num = 5#共训练5个周期
for epoch in range(epoch_num):
    for batch_idx, data in enumerate(dataloader):
        real_images, _ = data
        batch_size = real_images.shape[0]
        #训练判别器D
        labels = torch.ones(batch_size) # 真实数据的标签：1
        preds = dnet(Variable(real_images.type(Tensor))) #将真实数据喂给D网络
        outputs = preds.reshape(-1) # 转换成(batch_size,)维度
        dloss_real = criterion(outputs, labels.type(Tensor))
        dmean_real = outputs.sigmoid().mean() #计算判别器将多少真数据判别为真，仅用于输出显示
        
        noises = torch.randn(batch_size, latent_size, 1, 1) # [64, 64, 1, 1]
        fake_images = gnet(noises.type(Tensor)) # 生成假数据[64, 1, 28, 28]
        labels = torch.zeros(batch_size)    # 生成假数据的标签：0
        fake = fake_images.detach()     # 消除梯度,其实可以不用，因为最后更新的只是dnet的参数
        preds = dnet(fake)              # 将假数据喂给判别器
        outputs = preds.reshape(-1)     # 转换成(batch_size,)维度
        dloss_fake = criterion(outputs.type(Tensor), labels.type(Tensor))
        dmean_fake = outputs.sigmoid().mean() # 计算判别器将多少假数据判断为错的，仅用于输出显示
        
        dloss = dloss_real + dloss_fake #总的鉴别器损失为两者之和
        dnet.zero_grad()    # 梯度清零
        dloss.backward()    # 反向传播
        doptimizer.step()
        
        #训练生成器G
        labels = torch.ones(batch_size) # 在训练生成器G时，希望生成器的标签为1
        preds = dnet(fake_images)   # 让假数据通过鉴别网络
        outputs = preds.reshape(-1) # 转换成(batch_size,)维度
        gloss = criterion(outputs.type(Tensor), labels.type(Tensor))
        gmean_fake = outputs.sigmoid().mean() # 计算判别器将多少假数据判断为真，仅用于输出显示
        
        gnet.zero_grad()    # 梯度清零
        gloss.backward()    # 反向传播
        goptimizer.step()
        
        #输出本步训练结果
        # print('[{}/{}]'.format(epoch, epoch_num) + '[{}/{}]'.format(batch_idx, len(dataloader)) +
        #      '鉴别器G损失:{:g} 生成器D损失：{:g}'.format(dloss, gloss) + 
        #      '真数据判真比例：{:g} 假数据判真比例：{:g}/{:g}'.format(dmean_real, dmean_fake, gmean_fake))
        if batch_idx % 300 == 0:
            print('[{}/{}]'.format(epoch, epoch_num) + '[{}/{}]'.format(batch_idx, len(dataloader)) +
             '鉴别器G损失:{:g} 生成器D损失：{:g} '.format(dloss, gloss) + 
             '真数据判真比例：{:g} 假数据判假比例：{:g}，假数据判真比例：{:g}'.format(dmean_real, dmean_fake, gmean_fake))
            fake = gnet(fixed_noise) # 利用噪声生成假数据
            path = './dcgan_img/gpu{:02d}_batch{:03d}.png'.format(epoch, batch_idx)
            save_image(fake, path, normalize=False)
            
end = time.time()
print((end - start)/60) #输出结束时间(单位：分钟)

[0/5][0/938]鉴别器G损失:1.84623 生成器D损失：0.833955 真数据判真比例：0.282376 假数据判真比例：0.343563/0.46929
[0/5][300/938]鉴别器G损失:0.333063 生成器D损失：4.23638 真数据判真比例：0.958263 假数据判真比例：0.235754/0.0183889
[0/5][600/938]鉴别器G损失:0.370077 生成器D损失：1.88758 真数据判真比例：0.748114 假数据判真比例：0.0530615/0.175519
[0/5][900/938]鉴别器G损失:0.581111 生成器D损失：2.13298 真数据判真比例：0.939526 假数据判真比例：0.350107/0.159948
[1/5][0/938]鉴别器G损失:0.244237 生成器D损失：4.05243 真数据判真比例：0.98002 假数据判真比例：0.193074/0.0204833
[1/5][300/938]鉴别器G损失:0.774482 生成器D损失：1.68781 真数据判真比例：0.611245 假数据判真比例：0.159228/0.21547
[1/5][600/938]鉴别器G损失:0.130119 生成器D损失：3.45207 真数据判真比例：0.954396 假数据判真比例：0.0773588/0.0388486
[1/5][900/938]鉴别器G损失:0.247125 生成器D损失：3.65967 真数据判真比例：0.944922 假数据判真比例：0.159891/0.0342356
[2/5][0/938]鉴别器G损失:0.206971 生成器D损失：3.56086 真数据判真比例：0.960546 假数据判真比例：0.145074/0.0368723
[2/5][300/938]鉴别器G损失:0.166749 生成器D损失：3.16513 真数据判真比例：0.889824 假数据判真比例：0.0433403/0.0552686
[2/5][600/938]鉴别器G损失:0.882025 生成器D损失：2.01287 真数据判真比例：0.776345 假数据判真比例：0.379883/0.171522
[2/5][900/938]鉴别器G损失:0.199809 生成

## 保存模型

In [26]:
torch.save(dnet.state_dict(),'./dcgan_discriminator.pth')
torch.save(gnet.state_dict(),'./dcgan_generator.pth')